# Structured Report Generation with Ollama and Python

This notebook demonstrates how to build a structured report generation system using basic Python code and the Ollama API. Unlike complex frameworks like LangGraph, we'll implement all the necessary components from scratch, making it easier to understand and customize.

## What We'll Build

We'll create an AI-powered report generator that:
- Uses local language models via Ollama
- Performs web searches to gather information
- Generates structured reports with consistent formatting
- Implements tool calling and routing without complex frameworks

## Why Local Models?

- **Privacy**: Your data stays on your machine
- **Cost**: No API fees or usage limits
- **Control**: Full control over the model and generation process
- **Learning**: Better understanding of how these systems work

## Setup and Installation

First, let's install the required packages:

In [ ]:
# Install required packages
!pip install ollama pydantic duckduckgo-search

Make sure you have Ollama installed and running on your system. If not, download it from https://ollama.com/download

Pull the model we'll use:

In [ ]:
# Pull the model (this will take a few minutes the first time)
!ollama pull gemma3n

In [24]:
# Import required libraries
import json
import ollama
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from duckduckgo_search import DDGS
from datetime import datetime
import time

## Part 1: Structured Output with Pydantic

One of the key features we need is the ability to get structured, consistent outputs from our language model. We'll use Pydantic to define schemas and ensure the model returns data in the format we expect.

In [25]:
# Define the structure for a report section
class ReportSection(BaseModel):
    title: str = Field(description="The title of this section")
    content: str = Field(description="The main content of this section")
    key_points: List[str] = Field(description="Key takeaways from this section")
    sources: List[str] = Field(description="Sources used for this section", default=[])

# Define the overall report structure
class StructuredReport(BaseModel):
    title: str = Field(description="The main title of the report")
    executive_summary: str = Field(description="A brief executive summary")
    sections: List[ReportSection] = Field(description="The main sections of the report")
    conclusion: str = Field(description="The conclusion of the report")
    generated_at: str = Field(default_factory=lambda: datetime.now().isoformat())

# Example of how to use structured output with Ollama
def generate_structured_output(prompt: str, schema: BaseModel):
    """
    Generate structured output using Ollama with a Pydantic schema
    """
    response = ollama.chat(
        model='gemma3n',
        messages=[{
            'role': 'user',
            'content': prompt
        }],
        format=schema.model_json_schema()
    )
    
    # Parse the response
    return schema.model_validate_json(response['message']['content'])

In [26]:
# Test structured output generation
test_section = generate_structured_output(
    "Create a report section about the benefits of renewable energy",
    ReportSection
)
print(f"Title: {test_section.title}")
print(f"Content: {test_section.content[:200]}...")
print(f"Key Points: {test_section.key_points}")

Title: Benefits of Renewable Energy: A Sustainable Future
Content: ## Benefits of Renewable Energy: A Sustainable Future

Transitioning to renewable energy sources offers a multitude of benefits, spanning environmental, economic, and social dimensions.  This section ...
Key Points: ['Reduced greenhouse gas emissions and improved air quality.', 'Job creation, energy independence, and price stability.', 'Improved public health, energy access, and community empowerment.']


In [27]:
from IPython.display import display, Markdown

display(Markdown(f"### {test_section.title}\n\n{test_section.content}\n\n**Key Points:**\n" + "\n".join(f"- {kp}" for kp in test_section.key_points)))

### Benefits of Renewable Energy: A Sustainable Future

## Benefits of Renewable Energy: A Sustainable Future

Transitioning to renewable energy sources offers a multitude of benefits, spanning environmental, economic, and social dimensions.  This section outlines the key advantages of embracing renewable energy technologies.

**1. Environmental Advantages:**

*   **Reduced Greenhouse Gas Emissions:**  Renewable energy sources like solar, wind, hydro, and geothermal produce little to no greenhouse gas emissions during operation. This is crucial in mitigating climate change by reducing the concentration of heat-trapping gases in the atmosphere.  Compared to fossil fuels, renewables significantly lower our carbon footprint.
*   **Improved Air Quality:**  Unlike fossil fuel power plants, renewable energy facilities do not release harmful air pollutants such as sulfur dioxide, nitrogen oxides, and particulate matter. This leads to cleaner air, reducing respiratory illnesses and improving public health.
*   **Water Conservation:** Many renewable energy technologies, particularly solar and wind, require significantly less water than traditional fossil fuel power plants, which rely heavily on water for cooling. This is especially important in water-stressed regions.
*   **Reduced Environmental Impact:** Renewable energy projects generally have a smaller land footprint and less disruptive impact on ecosystems compared to fossil fuel extraction and transportation.  Careful planning and siting can further minimize environmental impacts.

**2. Economic Advantages:**

*   **Job Creation:** The renewable energy sector is a rapidly growing industry, creating numerous jobs in manufacturing, installation, maintenance, and research & development.  This offers significant economic opportunities and can stimulate local economies.
*   **Energy Independence & Security:**  Renewable energy sources are domestically available in most countries, reducing reliance on volatile global fossil fuel markets and enhancing energy security.  This protects economies from price fluctuations and geopolitical instability.
*   **Price Stability:**  Once a renewable energy facility is built, the fuel source (sun, wind, water) is free. This leads to more stable and predictable energy prices compared to fossil fuels, which are subject to market fluctuations.
*   **Economic Development:**  Investment in renewable energy projects can spur economic development in rural areas and create new business opportunities.  This can revitalize local economies and improve infrastructure.

**3. Social Advantages:**

*   **Improved Public Health:** Cleaner air and water resulting from renewable energy adoption directly contribute to improved public health outcomes and reduced healthcare costs.
*   **Energy Access:**  Renewable energy technologies, particularly off-grid solar solutions, can provide access to electricity for communities that lack access to traditional power grids, improving living standards and fostering economic development.
*   **Community Empowerment:**  Community-owned renewable energy projects can empower local communities, providing them with control over their energy future and generating local revenue.
*   **Sustainable Development:**  Renewable energy is a cornerstone of sustainable development, ensuring that future generations have access to clean and affordable energy resources.

**Conclusion:**

The benefits of renewable energy are undeniable.  By embracing these technologies, we can create a cleaner, more secure, and more prosperous future for all.  Continued investment in research, development, and deployment of renewable energy is essential to achieving a sustainable energy system.

**Key Points:**
- Reduced greenhouse gas emissions and improved air quality.
- Job creation, energy independence, and price stability.
- Improved public health, energy access, and community empowerment.

## Part 2: Web Search Integration

To create comprehensive reports, we need to gather information from the web. We'll implement a simple web search function that our AI can use.

In [37]:
def web_search(query: str, max_results: int = 5) -> List[Dict[str, str]]:
    """
    Perform a web search using DuckDuckGo
    
    Args:
        query: The search query
        max_results: Maximum number of results to return
    
    Returns:
        List of search results with title, body, and href
    """
    try:
        print(f"Searching for {query}...")
        results = DDGS().text(query, max_results=max_results)
        print("Search results: ", results)
        return [
            {
                "title": r.get("title", ""),
                "snippet": r.get("body", ""),
                "url": r.get("href", "")
            }
            for r in results
        ]
    except Exception as e:
        print(f"Search error: {e}")
        return []

# Test the web search function
search_results = web_search("artificial intelligence applications 2024")
for i, result in enumerate(search_results[:3]):
    print(f"\n{i+1}. {result['title']}")
    print(f"   {result['snippet'][:100]}...")
    print(f"   URL: {result['url']}")

Searching for artificial intelligence applications 2024...


/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_60944/3515036530.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)


Search results:  [{'title': 'The 2024 AI Index Report', 'href': 'https://hai.stanford.edu/ai-index/2024-ai-index-report', 'body': 'Welcome to the seventh edition of the AI Index report. The 2024 Index is our most comprehensive to date and arrives at an important moment when AI’s influence on society has never been more pronounced. This chapter studies trends in AI research and development.'}, {'title': 'Top 18 Artificial Intelligence (AI) Applications in 2024 - Rejolut', 'href': 'https://rejolut.com/blog/top-ai-applications/', 'body': 'As technology evolves at a breakneck pace, Artificial Intelligence (AI) stands out as a key catalyst, ushering us into a future where …'}, {'title': 'Top 10 AI Trends of 2024 - AI Magazine', 'href': 'https://aimagazine.com/articles/top-10-ai-trends-of-2024', 'body': 'Dec 11, 2024 · AI Magazine examines the top 10 AI trends of 2024 to understand the developments of this year and see what …'}, {'title': 'What’s next for AI in 2024 - MIT Technology Review',

## Part 3: Building a Simple Tool Calling System

Now let's implement a basic tool calling system that allows our LLM to use functions like web search.

In [51]:

# Available tools
AVAILABLE_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query string",
                    },
                    "max_results": {
                        "type": "integer",
                        "description": "Maximum number of results (default: 5)",
                    },
                },
                "required": ["query"],
            },
        },
    },
]


In [64]:
import copy
def llm_with_tools(prompt: str, include_tool_results: bool = True) -> str:
    """
    Process a prompt that may require tool usage using Ollama's native tool calling API.
    """
    
    # Use a supported tool-calling model
    model_name = "mistral-small3.2"
    
    prompt_prefix = """Look this up on the web with sources written in english preferably."""

    # Initial user message
    messages = [
        {'role': 'user', 'content': prompt}
    ]

    # Call Ollama with tools
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=AVAILABLE_TOOLS
    )
    print("RESPONSE")
    print(response)

    # Check if the model returned any tool calls
    tool_calls = response['message'].get('tool_calls', [])
    if not tool_calls:
        # No tool call, just return the model's answer
        return response['message']['content']

    # The model returned a tool call; handle the first one
    tool_call = tool_calls[0]
    # The structure is: {'function': {'name': ..., 'arguments': {...}}}
    tool_name = tool_call['function']['name']
    tool_args = tool_call['function'].get('arguments', {})

    # If arguments are a stringified JSON, parse them
    if isinstance(tool_args, str):
        try:
            tool_args = json.loads(tool_args)
        except Exception:
            pass

    # Execute the tool
    tool_result = web_search(tool_args['query'])
    print("TOOL RESULT")
    print(tool_result)

    # Prepare the tool response message for the model
    tool_response_message = {
        "role": "tool",
        "tool_call_id": tool_call.get("id", "tool_call_1"),
        "name": tool_name,
        "content": json.dumps(tool_result)
    }

    # Add the tool call and tool response to the conversation
    messages_with_tool = copy.deepcopy(messages)
    messages_with_tool.append({
        "role": "assistant",
        "tool_calls": [tool_call]
    })
    messages_with_tool.append(tool_response_message)

    if include_tool_results:
        # Ask the model to generate a final answer with the tool result
        final_response = ollama.chat(
            model=model_name,
            messages=messages_with_tool
        )
        return final_response['message']['content']
    else:
        # Just return the tool result as JSON
        return json.dumps(tool_result.dict(), indent=2)

In [65]:
# Test the tool calling system
result = llm_with_tools("who won the nba 2025?")
print(result)

RESPONSE
{'model': 'mistral-small3.2', 'created_at': '2025-07-30T11:55:30.780481Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'web_search', 'arguments': {'query': 'nba 2025 champion'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 3312361500, 'load_duration': 55617208, 'prompt_eval_count': 591, 'prompt_eval_duration': 1532043375, 'eval_count': 18, 'eval_duration': 1722916083}
Searching for nba 2025 champion...


/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_60944/3515036530.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)


Search results:  [{'title': '2025 NBA Finals | NBA.com', 'href': 'https://www.nba.com/playoffs/2025/nba-finals', 'body': 'Thunder GM Sam Presti, who wins the NBA championship in a 2nd Finals with OKC, joins NBA TV to discuss the nearly 20-year journey. Chasing History: Thunder Make History! In the …'}, {'title': '2025 NBA Finals - Pacers vs. Thunder | Basketball-Reference.com', 'href': 'https://www.basketball-reference.com/playoffs/2025-nba-finals-pacers-vs-thunder.html', 'body': 'League Champion: Oklahoma City Thunder Finals MVP: Shai Gilgeous-Alexander (30.3 / 4.6 / 5.6) 2025 Playoff Leaders: PTS: Shai Gilgeous-Alexander (688) TRB: Karl-Anthony Towns …'}, {'title': 'NBA Championships 2025: Complete List Of Awards And Winners', 'href': 'https://nbafiles.com/news/nba-championships-2025-list-of-winners/', 'body': 'Jun 23, 2025 · The NBA Championships 2025 ended with an intense finale between OKC Thunder and Indiana Pacers. Read this article to know the full list of winners.'}, {'title':

## Part 4: Report Generation Pipeline

Now let's build the main report generation pipeline that combines all our components.

In [66]:
class ReportGenerator:
    def __init__(self, model_name: str = 'mistral-small3.2'):
        self.model_name = model_name
        
    def research_topic(self, topic: str, num_searches: int = 3) -> List[Dict[str, Any]]:
        """
        Research a topic by performing multiple web searches
        """
        # Generate search queries
        query_prompt = f"""
        Generate {num_searches} different search queries to research the topic: "{topic}"
        Make the queries specific and diverse to cover different aspects.
        Return as a JSON list of strings.
        """
        
        response = ollama.chat(
            model=self.model_name,
            messages=[{'role': 'user', 'content': query_prompt}]
        )
        
        try:
            queries = json.loads(response['message']['content'])
        except:
            # Fallback to simple query generation
            queries = [
                topic,
                f"{topic} latest developments",
                f"{topic} challenges and opportunities"
            ]
        
        # Perform searches
        all_results = []
        for query in queries[:num_searches]:
            print(f"Searching for: {query}")
            results = web_search(query, max_results=3)
            all_results.extend(results)
            time.sleep(1)  # Be nice to the search API
        
        return all_results
    
    def generate_section(self, topic: str, section_title: str, research_data: List[Dict]) -> ReportSection:
        """
        Generate a specific section of the report
        """
        prompt = f"""
        Create a report section about: {section_title}
        Topic: {topic}
        
        Use the following research data:
        {json.dumps(research_data[:5], indent=2)}
        
        Make the content informative and well-structured.
        Include specific examples and data points from the research.
        """
        
        return generate_structured_output(prompt, ReportSection)
    
    def generate_report(self, topic: str, sections: List[str] = None) -> StructuredReport:
        """
        Generate a complete structured report on a topic
        """
        print(f"Generating report on: {topic}")
        
        # Default sections if not provided
        if sections is None:
            sections = [
                "Overview and Current State",
                "Key Developments and Trends",
                "Challenges and Opportunities",
                "Future Outlook"
            ]
        
        # Research the topic
        print("\nResearching topic...")
        research_data = self.research_topic(topic)
        
        # Generate sections
        report_sections = []
        for section_title in sections:
            print(f"\nGenerating section: {section_title}")
            section = self.generate_section(topic, section_title, research_data)
            report_sections.append(section)
        
        # Generate executive summary and conclusion
        summary_prompt = f"""
        Based on the following report sections about "{topic}", 
        create an executive summary that highlights the key findings.
        
        Sections:
        {json.dumps([s.dict() for s in report_sections], indent=2)}
        
        Make it concise but comprehensive.
        """
        
        summary_response = ollama.chat(
            model=self.model_name,
            messages=[{'role': 'user', 'content': summary_prompt}]
        )
        
        conclusion_prompt = f"""
        Based on the report about "{topic}", write a strong conclusion that:
        1. Summarizes the main findings
        2. Provides actionable insights
        3. Suggests areas for future research or attention
        """
        
        conclusion_response = ollama.chat(
            model=self.model_name,
            messages=[{'role': 'user', 'content': conclusion_prompt}]
        )
        
        # Create the final report
        report = StructuredReport(
            title=f"Comprehensive Report: {topic}",
            executive_summary=summary_response['message']['content'],
            sections=report_sections,
            conclusion=conclusion_response['message']['content']
        )
        
        return report

In [67]:
# Helper function to display report nicely
def display_report(report: StructuredReport):
    print(f"\n{'='*80}")
    print(f"\n{report.title}")
    print(f"\nGenerated: {report.generated_at}")
    print(f"\n{'='*80}")
    
    print(f"\n\nEXECUTIVE SUMMARY")
    print(f"\n{report.executive_summary}")
    
    for section in report.sections:
        print(f"\n\n{'-'*60}")
        print(f"\n{section.title.upper()}")
        print(f"\n{section.content}")
        
        if section.key_points:
            print(f"\n**Key Points:**")
            for point in section.key_points:
                print(f"  • {point}")
    
    print(f"\n\n{'-'*60}")
    print(f"\nCONCLUSION")
    print(f"\n{report.conclusion}")
    print(f"\n{'='*80}\n")

## Part 5: Example Applications

Let's use our report generator to create some example reports.

In [68]:
# Example 1: Technology Report
generator = ReportGenerator()
tech_report = generator.generate_report(
    "Artificial Intelligence in Healthcare 2024",
    sections=[
        "Current Applications in Healthcare",
        "Recent Breakthroughs and Innovations",
        "Regulatory and Ethical Considerations",
        "Market Analysis and Future Trends"
    ]
)

display_report(tech_report)

Generating report on: Artificial Intelligence in Healthcare 2024

Researching topic...
Searching for: Artificial Intelligence in Healthcare 2024
Searching for Artificial Intelligence in Healthcare 2024...
Search results:  [{'title': 'Camping Lido Valderice', 'href': 'http://campinglidovalderice.it/', 'body': 'Per una vacanza in campeggio in Sicilia, Camping Lido Valderice offre un pernottamento tranquillo e piacevole davanti al mare con case mobili (bungalow), tende e camper. Via della …'}, {'title': 'CAMPING LIDO VALDERICE : Prezzi e Recensioni (2025)', 'href': 'https://www.tripadvisor.it/Hotel_Review-g635633-d4801654-Reviews-Camping_Lido_Valderice-Valderice_Province_of_Trapani_Sicily.html', 'body': 'Quali sono alcuni dei servizi disponibili presso Camping Lido Valderice? Alcuni tra i servizi offerti più richiesti includono connessione Wi-Fi gratuita, ristorante e lounge. Vedi tutti i servizi della …'}, {'title': 'Camping Lido Valderice | ACSI - ACSI Eurocampings', 'href': 'https://ww

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_60944/3515036530.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)


Searching for: Artificial Intelligence in Healthcare 2024 latest developments
Searching for Artificial Intelligence in Healthcare 2024 latest developments...


/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_60944/3515036530.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)


Search results:  [{'title': 'Check or delete your Chrome browsing history - Google Help', 'href': 'https://support.google.com/chrome/answer/95589?hl=en&co=GENIE.Platform=Desktop', 'body': 'Websites you’ve visited are recorded in your browsing history. You can check or delete your browsing history, and find related searches in Chrome. You can also resume browsing …'}, {'title': 'Delete your activity - Computer - Google Account Help', 'href': 'https://support.google.com/accounts/answer/465?hl=en&co=GENIE.Platform=Desktop', 'body': 'On your computer, go to your Google Account. At the left, click Data & privacy. Under "History settings," click an activity or history setting you want to auto-delete. Click Auto-delete. …'}, {'title': 'Access & control activity in your account - Google Help', 'href': 'https://support.google.com/accounts/answer/7028918?hl=en&co=GENIE.Platform=Desktop', 'body': 'Under "History settings," click My Activity. To access your activity: Browse your activity, organize

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_60944/3515036530.py:14: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(query, max_results=max_results)


Search results:  [{'title': 'Artificial intelligence in healthcare: Opportunities and c…', 'href': 'https://www.researchgate.net/publication/376709634_Artificial_intelligence_in_healthcare_Opportunities_and_challenges', 'body': 'Dec 20, 2023 · The development of Artificial Intelligence (AI) in healthcare has had a significant impact on healthcare. …'}, {'title': 'Artificial Intelligence in Health Care: Opportunities, Challenges…', 'href': 'https://pubmed.ncbi.nlm.nih.gov/39466089/', 'body': 'A comprehensive, collective approach to navigating the challenges of bias, privacy, and ethical considerations presented …'}, {'title': 'Opportunities and challenges of artificial intelligence and distrib…', 'href': 'https://www.sciencedirect.com/science/article/pii/S0933365724000216', 'body': 'Mar 1, 2024 · A discernible pattern emerges in the provided comprehensive analysis addressing the opportunities …'}]

Generating section: Current Applications in Healthcare

Generating section: Recent Breakt

/Users/greatmaster/miniconda3/envs/oreilly-llama3/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/greatmaster/miniconda3/envs/oreilly-llama3/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/greatmaster/miniconda3/envs/oreilly-llama3/lib/python3.10/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict



Comprehensive Report: Artificial Intelligence in Healthcare 2024

Generated: 2025-07-30T13:03:44.619469



EXECUTIVE SUMMARY

Here’s a concise yet comprehensive summary of the key points across the four documents on **AI in Healthcare (2024)**:

### **1. Applications & Benefits**
- **Diagnostics & Imaging**: AI improves accuracy in detecting diseases (e.g., cancer, cardiovascular issues) from medical images.
- **Drug Discovery**: Accelerates development by analyzing molecular data, reducing costs and time.
- **Personalized Medicine**: Tailors treatments using patient data (genomics, lifestyle, history).
- **Robotic Surgery**: Enhances precision, reduces recovery times, and minimizes complications.
- **Virtual Assistants**: Provides 24/7 patient support, appointment scheduling, and medication reminders.
- **Predictive Analytics**: Identifies future health risks for proactive care.

### **2. Market Trends & Growth**
- **Drivers**: Rising healthcare costs, data explosion (EHRs, wearable

In [ ]:
# Example 2: Market Research Report
market_report = generator.generate_report(
    "Electric Vehicle Market Analysis",
    sections=[
        "Market Size and Growth Projections",
        "Key Players and Competition",
        "Technology Trends and Innovations",
        "Investment Opportunities and Risks"
    ]
)

# Save to file
with open('ev_market_report.json', 'w') as f:
    json.dump(market_report.dict(), f, indent=2)
print("Report saved to ev_market_report.json")

## Part 6: Advanced Features

Let's add some advanced features to make our report generator more robust.

In [ ]:
# Retry logic for failed generations
def generate_with_retry(func, max_retries=3, *args, **kwargs):
    """
    Retry a function call if it fails
    """
    for attempt in range(max_retries):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)  # Exponential backoff

# Section validation
def validate_section(section: ReportSection) -> bool:
    """
    Validate that a section meets quality standards
    """
    if len(section.content) < 100:
        print("Warning: Section content too short")
        return False
    
    if len(section.key_points) < 2:
        print("Warning: Too few key points")
        return False
    
    return True

# Enhanced report generator with validation
class EnhancedReportGenerator(ReportGenerator):
    def generate_section(self, topic: str, section_title: str, research_data: List[Dict]) -> ReportSection:
        """
        Generate a section with validation and retry logic
        """
        section = generate_with_retry(
            super().generate_section,
            max_retries=3,
            topic=topic,
            section_title=section_title,
            research_data=research_data
        )
        
        # Validate and regenerate if needed
        if not validate_section(section):
            print(f"Regenerating section: {section_title}")
            enhanced_prompt = f"""
            Create a DETAILED report section about: {section_title}
            Topic: {topic}
            
            Requirements:
            - Content must be at least 200 words
            - Include at least 3 key points
            - Use specific examples from the research data
            
            Research data:
            {json.dumps(research_data[:5], indent=2)}
            """
            section = generate_structured_output(enhanced_prompt, ReportSection)
        
        return section

In [ ]:
# Custom report templates
REPORT_TEMPLATES = {
    "technical": {
        "sections": [
            "Technical Overview",
            "Architecture and Implementation",
            "Performance Analysis",
            "Security Considerations",
            "Integration Guidelines"
        ]
    },
    "business": {
        "sections": [
            "Market Overview",
            "Competitive Analysis",
            "Revenue Models",
            "Risk Assessment",
            "Strategic Recommendations"
        ]
    },
    "research": {
        "sections": [
            "Literature Review",
            "Methodology",
            "Key Findings",
            "Discussion",
            "Future Research Directions"
        ]
    }
}

def generate_templated_report(topic: str, template: str = "technical"):
    """
    Generate a report using a predefined template
    """
    if template not in REPORT_TEMPLATES:
        raise ValueError(f"Unknown template: {template}")
    
    generator = EnhancedReportGenerator()
    sections = REPORT_TEMPLATES[template]["sections"]
    
    return generator.generate_report(topic, sections)

In [ ]:
# Example: Generate a technical report
technical_report = generate_templated_report(
    "Quantum Computing Applications",
    template="technical"
)

# Export to markdown
def export_to_markdown(report: StructuredReport, filename: str):
    """
    Export report to markdown format
    """
    with open(filename, 'w') as f:
        f.write(f"# {report.title}\n\n")
        f.write(f"*Generated: {report.generated_at}*\n\n")
        f.write(f"## Executive Summary\n\n{report.executive_summary}\n\n")
        
        for section in report.sections:
            f.write(f"## {section.title}\n\n")
            f.write(f"{section.content}\n\n")
            
            if section.key_points:
                f.write("### Key Points\n\n")
                for point in section.key_points:
                    f.write(f"- {point}\n")
                f.write("\n")
        
        f.write(f"## Conclusion\n\n{report.conclusion}\n")

export_to_markdown(technical_report, "quantum_computing_report.md")
print("Report exported to quantum_computing_report.md")

## Conclusion and Next Steps

We've built a complete structured report generation system using basic Python and Ollama. Here's what we accomplished:

1. **Structured Output**: Used Pydantic schemas to ensure consistent report formatting
2. **Web Search Integration**: Added the ability to research topics using DuckDuckGo
3. **Tool Calling**: Implemented a simple but effective tool calling system
4. **Report Pipeline**: Created a complete pipeline for generating multi-section reports
5. **Advanced Features**: Added validation, retry logic, and templates

### Ideas for Extension:

- Add more tools (Wikipedia search, arxiv papers, news APIs)
- Implement citation tracking and source verification
- Add support for generating charts and visualizations
- Create a web interface for the report generator
- Add support for different output formats (PDF, HTML)
- Implement collaborative report generation with multiple LLMs
- Add fact-checking and consistency validation

### Key Takeaways:

- Local LLMs can be powerful tools for structured content generation
- You don't need complex frameworks to build useful AI applications
- Combining web search with LLMs creates more accurate and up-to-date content
- Structured output ensures consistency and makes integration easier

Happy report generating! 🚀